In [1]:
##数据预处理
# 这是一个花朵识别的demo，数据集包含了5种花朵，数据都是脏数据 需要预处理
# 这里用到的数据集:http://download.tensorflow.org/example_images/flower_photos.tgz
# 注意这个数据集处理需要大概16G+的内存 否则会出现MemError，这一问题可通过减少图片数量来解决

import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# 原始输入数据的目录，这个目录下有5个子目录，每个子目录底下保存这属于该类别的所有图片。
INPUT_DATA = '/home/se7ven/Desktop/Inception-v3/datasets/flower_photos'
# 输出文件地址。我们将整理后的图片数据通过numpy的格式保存。
OUTPUT_FILE = '/home/se7ven/Desktop/Inception-v3/datasets/flower_processed_data.npy'

# 测试数据和验证数据比例。
VALIDATION_PERCENTAGE = 10
TEST_PERCENTAGE = 10


# 读取数据并将数据分割成训练数据、验证数据和测试数据。
def create_image_lists(sess, testing_percentage, validation_percentage):
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True
    
    # 初始化各个数据集。
    training_images = []
    training_labels = []
    validation_images = []
    validation_labels = []
    testing_images = []
    testing_labels = []
    current_label = 0
    
    # 读取所有的子目录。
    for sub_dir in sub_dirs:
        if is_root_dir:        # sub_dirs第一个元素为根目录路径
            is_root_dir = False
            continue

        # 获取一个子目录中所有的图片文件。
        extensions = ['jpg']
        file_list = []
        dir_name = os.path.basename(sub_dir)    # 返回path最后的文件名，/或\结尾则返回空值
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.' + extension)     #  ../../../datasets/flower_photos\daisy\*.jpg
            file_list.extend(glob.glob(file_glob))    # 匹配所有符合条件文件，并以list返回
        if not file_list: continue
        print("processing: %s. total %d images."% (dir_name, len(file_list)))
        
        i = 0
        # 处理图片数据。
        for file_name in file_list:
            i += 1
            # 读取并解析图片，将图片转化为299*299以方便inception-v3模型来处理。
            image_raw_data = gfile.FastGFile(file_name, 'rb').read()
            #解码
            image = tf.image.decode_jpeg(image_raw_data)
            #如果图片格式不是float32则转为float32
            if image.dtype != tf.int32:
                image = tf.image.convert_image_dtype(image, dtype=tf.int32)
            #将图片源数据转为299*299
            image = tf.image.resize_images(image, [299, 299])
            #得到此图片的数据
            image_value = sess.run(image)
            
            #按概率随机分到三个数据集中
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(image_value)
                validation_labels.append(current_label)
            elif chance < (testing_percentage + validation_percentage):
                testing_images.append(image_value)
                testing_labels.append(current_label)
            else:
                training_images.append(image_value)
                training_labels.append(current_label)
            if i % 100 == 0:
                print(i, "images processed.")
        #处理完此种品种就将标签+1
        current_label += 1
    
    # 将训练数据随机打乱以获得更好的训练效果。
    # 这里的get_state记录下数组被打乱的操作，set_state读取信息后可进行同样的操作，保证labels不变
    # 用法参见https://blog.csdn.net/mr_muli/article/details/80913915
    state = np.random.get_state()
    np.random.shuffle(training_images)
    np.random.set_state(state)
    np.random.shuffle(training_labels)
    
    return np.asarray([training_images, training_labels,
                       validation_images, validation_labels,
                       testing_images, testing_labels])


# 运行数据处理过程
with tf.Session() as sess:
    processed_data = create_image_lists(sess, TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
    # 通过numpy格式保存处理后的数据。
    np.save(OUTPUT_FILE, processed_data)

W0829 16:57:43.250150 140503674373952 deprecation.py:323] From <ipython-input-1-ac1fa7eb1403>:56: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.


processing: roses. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: daisy. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: tulips. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: dandelion. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: sunflowers. total 300 images.
100 images processed.
200 images processed.
300 images processed.


In [1]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import tensorflow.contrib.slim as slim
# 加载通过TensorFlow-Slim定义好的inception_v3模型。
import tensorflow.contrib.slim.python.slim.nets.inception_v3 as inception_v3

# 1. 定义训练过程中将要使用到的常量。
# 处理好之后的数据文件。
INPUT_DATA = '/home/se7ven/Desktop/Inception-v3/datasets/flower_processed_data.npy'
# 保存训练好的模型的路径。这里可以将使用新数据训练得到的模型保存下来，如果计算充足，
# 还可以在训练完最后的全连接层之后再训练所有的网络层，使模型更贴近新数据。
TRAIN_FILE = 'train_dir/model'
# 谷歌提供的训练好的模型文件地址。因为GitHub无法保存大于100M的文件，所以
# 在运行时需要先自行从Google下载inception_v3.ckpt文件。
CKPT_FILE = '/home/se7ven/Desktop/Inception-v3/datasets/inception_v3.ckpt'

# 定义训练中使用的参数。
LEARNING_RATE = 0.0001
STEPS = 500
BATCH = 32
N_CLASSES = 5

# 不需要从谷歌训练好的模型中加载的参数。这里就是最后的全连接层。这里给出的是参数的前缀
CHECKPOINT_EXCLUDE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogits'
# 需要训练的网络层参数明层，在fine-tuning的过程中就是最后的全联接层。这里给出的是参数的前缀
TRAINABLE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogit'


# 2. 获取所有需要从谷歌训练好的模型中加载的参数。
def get_tuned_variables():
    ##不需要加载的范围
    exclusions = [scope.strip() for scope in CHECKPOINT_EXCLUDE_SCOPES.split(',')]
    # 初始化需要加载的参数
    variables_to_restore = []
    
    # 遍历模型中所有的参数，然后判断是否需要从加载列表中移除。
    for var in slim.get_model_variables():
        # 先指定为不需要移除
        excluded = False
        # 遍历exclusions，如果在exclusions中，就指定为需要移除
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        # 如果遍历完后还是不需要移除，就把参数加到列表里        
        if not excluded:
            variables_to_restore.append(var)
    return variables_to_restore


# 3. 获取所有需要训练的变量列表。
def get_trainable_variables():    
    scopes = [scope.strip() for scope in TRAINABLE_SCOPES.split(',')]
    variables_to_train = []
    
    # 枚举所有需要训练的参数前缀，并通过这些前缀找到所有需要训练的参数。
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    return variables_to_train


# 4. 定义训练过程
def main():
    
    # 这里是因为numpy的版本更新了 如果不allow_pickle=True会报错，于是先打开，再改回来
    np_load_old = np.load
    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
    
    # 0. 加载预处理好的数据。
    processed_data = np.load(INPUT_DATA)
    
    # restore np.load for future normal usage
    np.load = np_load_old
    
    training_images = processed_data[0]
    n_training_example = len(training_images)
    training_labels = processed_data[1]
    
    validation_images = processed_data[2]
    validation_labels = processed_data[3]
    
    testing_images = processed_data[4]
    testing_labels = processed_data[5]
    print("%d training examples, %d validation examples and %d testing examples." % (
        n_training_example, len(validation_labels), len(testing_labels)))

    # 1. 定义inception-v3的输入输出的数据格式
    images = tf.placeholder(tf.float32, [None, 299, 299, 3], name='input_images')
    labels = tf.placeholder(tf.int64, [None], name='labels')
    
    # 2. 定义前向传播、损失函数、反向传播
    # 定义inception-v3模型。因为谷歌给出的只有模型参数取值，所以这里
    # 需要在这个代码中定义inception-v3的模型结构。虽然理论上需要区分训练和
    # 测试中使用到的模型，也就是说在测试时应该使用is_training=False，但是
    # 因为预先训练好的inception-v3模型中使用的batch normalization参数与
    # 新的数据会有出入，所以这里直接使用同一个模型来做测试。
    # 可修改num_class = None
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, _ = inception_v3.inception_v3(
            images, num_classes=N_CLASSES, is_training=True)
    
    # 获取需要训练的变量
    trainable_variables = get_trainable_variables()    # 自定义函数，获取需要训练的变量
    # 定义损失
    tf.losses.softmax_cross_entropy(
        tf.one_hot(labels, N_CLASSES), logits, weights=1.0)
    total_loss = tf.losses.get_total_loss()
    
    # 定义训练过程(反向传播)
    train_step = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(total_loss)
    
    # 计算正确率。
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(logits, 1), labels)
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                
    # 定义加载Google训练好的Inception-v3模型的Saver。
    # (重新定义load_fn函数，从文件中获取参数，获取指定的变量，忽略缺省值)
    load_fn = slim.assign_from_checkpoint_fn(CKPT_FILE,get_tuned_variables(),ignore_missing_vars=True)
    
    # 3. 建立会话，训练模型
    # 定义保存新的训练好的模型的函数
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 初始化没有加载进来的变量。注意这个过程要在加载模型之前，
        # 否则初始化过程会将已经加载好的变量重新赋值。
        init = tf.global_variables_initializer()
        sess.run(init)
        
        # 加载谷歌已经训练好的模型。
        print('Loading tuned variables from %s' % CKPT_FILE)
        load_fn(sess)
            
        start = 0
        end = BATCH
        for i in range(STEPS):
            # 运行训练过程，这里不会更新全部参数，只会更新指定部分的参数
            _, loss = sess.run([train_step, total_loss], feed_dict={
                images: training_images[start:end], 
                labels: training_labels[start:end]})

            # 间断地保存模型，并在验证集上验证
            if i % 80 == 0 or i + 1 == STEPS:
                saver.save(sess, TRAIN_FILE, global_step=i)
                validation_accuracy = sess.run(evaluation_step, feed_dict={
                    images: validation_images, labels: validation_labels})
                print('Step %d: Training loss is %.1f Validation accuracy = %.1f%%' % (
                    i, loss, validation_accuracy * 100.0))
                         
            # 更新起始和末尾,获取下一个batch
            start = end
            if start == n_training_example:
                start = 0
            end = start + BATCH
            if end > n_training_example:
                end = n_training_example
            
        # 训练完了在测试集上测试正确率
        test_accuracy = sess.run(evaluation_step, feed_dict={
            images: testing_images, labels: testing_labels})
        print('Final test accuracy = %.1f%%' % (test_accuracy * 100))
        
        
if __name__ == '__main__':
    main()

#最后的准确度有些低， 因为只取了比较少的数据进行训练

1206 training examples, 160 validation examples and 134 testing examples.


W0829 17:15:00.145314 140491283457856 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0829 17:15:01.903965 140491283457856 deprecation.py:506] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0829 17:15:07.319084 140491283457856 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensor

Loading tuned variables from /home/se7ven/Desktop/Inception-v3/datasets/inception_v3.ckpt
Step 0: Training loss is 2.0 Validation accuracy = 20.0%
Step 30: Training loss is 1.9 Validation accuracy = 21.3%
Step 60: Training loss is 1.6 Validation accuracy = 41.9%
Step 90: Training loss is 1.0 Validation accuracy = 79.4%
Step 120: Training loss is 0.5 Validation accuracy = 85.0%


W0829 17:16:30.200701 140491283457856 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Step 150: Training loss is 0.3 Validation accuracy = 88.7%
Step 180: Training loss is 0.2 Validation accuracy = 90.0%
Step 210: Training loss is 0.2 Validation accuracy = 90.0%
Step 240: Training loss is 0.2 Validation accuracy = 87.5%
Step 270: Training loss is 0.2 Validation accuracy = 90.0%
Step 299: Training loss is 0.2 Validation accuracy = 92.5%
Final test accuracy = 89.6%
